In [9]:
!pip install beautifulsoup4


In [10]:
!pip install selenium

In [46]:
#Importing librairies

# import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)



import itertools

# Matplotlib library to plot the charts
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib as mpl

# Library for the statistic data vizualisation
import seaborn as sns


# Scikit-learn library: 
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import sklearn.metrics as metrics

from sklearn.model_selection import GridSearchCV

#Import time related library
import datetime
import requests
import time

#Import web scrap related library
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

C:\Users\annab\Anaconda3\envs\ads08\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# List out the station codes which needed weather data to be downloaded. 

In [12]:
from pandas import read_excel
my_sheet = 'Sheet9' # change it to the sheet name
file_name = 'Weather data from Weather Underground.xlsx' # change it to the name of the excel file
df_weather = read_excel(file_name, sheet_name = my_sheet)
df_weather.head() # shows headers with top 5 rows

,Pollutant Sensor Post Code,Weather Sensor Station Name,Code,New Post Code,Code_new
0,BN435DU,GATWICK AIRPORT STATION,EGKK,BN435DU,EGKK
1,RM70RJ,LONDON CITY AIRPORT STATION,EGLC,RM70RJ,EGLC
2,IG110UG,LONDON CITY AIRPORT STATION,EGLC,IG110UG,EGLC
3,DA176AA,LONDON CITY AIRPORT STATION,EGLC,DA176AA,EGLC
4,DA184HQ,LONDON CITY AIRPORT STATION,EGLC,DA184HQ,EGLC


In [6]:
code = df_weather.Code.unique()
# code = code[1:3]
code

array(['EGKK', 'EGLC', 'EGHI', 'EGLF', 'EGMD', 'EGLL', 'EGMC', nan],
      dtype=object)

In [13]:
# Given a table soup, returns all the headers
def get_table_headers(table):
   
    headers = []
    for th in table.find("tr").find_all("th"):
        headers.append(th.text.strip())
    return headers

def get_table_rows(table):
    """Given a table, returns all its rows"""
    rows = []
    for tr in table.find_all("tr")[1:]:
        cells = []
        # grab all td tags in this table row
        tds = tr.find_all("td")
        if len(tds) == 0:
            # if no td tags, search for th tags
            # can be found especially in wikipedia tables below the table
            ths = tr.find_all("th")
            for th in ths:
                cells.append(th.text.strip())
        else:
            # use regular td tags
            for td in tds:
                cells.append(td.text.strip())
        rows.append(cells)
    return rows

## The complete code for scrap weather data!

In [15]:
year = 2020
month = 12
days = 32

In [16]:
import datetime
import requests
import time

def print_weather(c,year,month,days):
    df = pd.DataFrame()

    for day in range(1,days):
        try: 
            print(day)
            url = "https://www.wunderground.com/history/daily/gb/gatwick-airport/"+str(c)+"/date/"+str(year)+"-"+str(month)+"-"+str(day)
            print(url)
            driver = webdriver.Chrome("C:\\Users\\annab\\Downloads\\chromedriver_win32.exe")
            driver.delete_all_cookies()
            driver.get(url)
            time.sleep(10)
            print(driver.title)
            print('driver current url', driver.current_url)


    #         soup = BeautifulSoup(driver.page_source, 'html.parser')
            soup = BeautifulSoup(driver.page_source)

            gdp_table_new = ""
            gdp_table_new = soup.find("table", attrs={"aria-labelledby": "History observation"})

            col = get_table_headers(gdp_table_new)

            # convert table's rows to be a dataframe.
            df_temp = pd.DataFrame(get_table_rows(gdp_table_new), columns = col )
            df_temp=df_temp.dropna()
            df_temp['Date'] = datetime.date(year,month,day)
    #         df_temp.head(3)
            first_col = df_temp.pop('Date')
            df_temp.insert(0, 'Date', first_col)
            df_temp['Station_Code'] = c
            display(df_temp.head(3))
            frames = [df,df_temp]
            df=pd.concat(frames)
            display(df.head(3))
#             time.sleep(10)
            driver.quit()    
        
        except: 
            continue
    df.to_csv(str(year)+'weather data-'+str(month)+'-'+str(c)+'.csv')
    
for c in code:  
    print(c)
    print_weather(c,year,month,days)

EGTE
1
https://www.wunderground.com/history/daily/gb/gatwick-airport/EGTE/date/2020-12-1
Exeter, United Kingdom Weather History | Weather Underground
driver current url https://www.wunderground.com/history/daily/gb/gatwick-airport/EGTE/date/2020-12-1
2
https://www.wunderground.com/history/daily/gb/gatwick-airport/EGTE/date/2020-12-2
Exeter, United Kingdom Weather History | Weather Underground
driver current url https://www.wunderground.com/history/daily/gb/gatwick-airport/EGTE/date/2020-12-2
3
https://www.wunderground.com/history/daily/gb/gatwick-airport/EGTE/date/2020-12-3
Exeter, United Kingdom Weather History | Weather Underground
driver current url https://www.wunderground.com/history/daily/gb/gatwick-airport/EGTE/date/2020-12-3
4
https://www.wunderground.com/history/daily/gb/gatwick-airport/EGTE/date/2020-12-4
Exeter, United Kingdom Weather History | Weather Underground
driver current url https://www.wunderground.com/history/daily/gb/gatwick-airport/EGTE/date/2020-12-4
5
https://w

,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Station_Code
0,2020-12-01,12:20 AM,37 F,34 F,87 %,VAR,2 mph,0 mph,29.99 in,0.0 in,Fair,EGCC
1,2020-12-01,12:50 AM,36 F,34 F,93 %,W,5 mph,0 mph,29.99 in,0.0 in,Fair,EGCC
2,2020-12-01,1:20 AM,36 F,34 F,93 %,W,5 mph,0 mph,29.99 in,0.0 in,Fair,EGCC


,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Station_Code
0,2020-12-01,12:20 AM,37 F,34 F,87 %,VAR,2 mph,0 mph,29.99 in,0.0 in,Fair,EGCC
1,2020-12-01,12:50 AM,36 F,34 F,93 %,W,5 mph,0 mph,29.99 in,0.0 in,Fair,EGCC
2,2020-12-01,1:20 AM,36 F,34 F,93 %,W,5 mph,0 mph,29.99 in,0.0 in,Fair,EGCC


2
https://www.wunderground.com/history/daily/gb/gatwick-airport/EGCC/date/2020-12-2
Manchester, United Kingdom Weather History | Weather Underground
driver current url https://www.wunderground.com/history/daily/gb/gatwick-airport/EGCC/date/2020-12-2


,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Station_Code
0,2020-12-02,12:20 AM,43 F,39 F,87 %,VAR,3 mph,0 mph,29.96 in,0.0 in,Cloudy,EGCC
1,2020-12-02,12:50 AM,43 F,39 F,87 %,VAR,3 mph,0 mph,29.96 in,0.0 in,Cloudy,EGCC
2,2020-12-02,1:20 AM,43 F,39 F,87 %,VAR,2 mph,0 mph,29.93 in,0.0 in,Cloudy,EGCC


...

,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Station_Code
0,2020-12-30,12:20 AM,36 F,28 F,75 %,NNW,12 mph,0 mph,29.49 in,0.0 in,Fair,EGPU
1,2020-12-30,12:50 AM,36 F,30 F,81 %,NNW,12 mph,0 mph,29.49 in,0.0 in,Fair,EGPU
2,2020-12-30,1:20 AM,36 F,32 F,87 %,NNW,10 mph,0 mph,29.49 in,0.0 in,Fair,EGPU


,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Station_Code
0,2020-12-01,12:20 AM,41 F,34 F,76 %,WNW,6 mph,0 mph,30.25 in,0.0 in,Fair,EGPU
1,2020-12-01,12:50 AM,41 F,34 F,76 %,W,5 mph,0 mph,30.25 in,0.0 in,Fair,EGPU
2,2020-12-01,1:20 AM,39 F,34 F,81 %,W,5 mph,0 mph,30.25 in,0.0 in,Fair,EGPU


31
https://www.wunderground.com/history/daily/gb/gatwick-airport/EGPU/date/2020-12-31
Isle Of Tiree, United Kingdom Weather History | Weather Underground
driver current url https://www.wunderground.com/history/daily/gb/gatwick-airport/EGPU/date/2020-12-31


,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Station_Code
0,2020-12-31,12:20 AM,39 F,37 F,93 %,NNW,18 mph,0 mph,29.58 in,0.0 in,Mostly Cloudy,EGPU
1,2020-12-31,12:50 AM,39 F,36 F,87 %,N,20 mph,0 mph,29.58 in,0.0 in,Partly Cloudy,EGPU
2,2020-12-31,1:20 AM,41 F,36 F,81 %,N,17 mph,0 mph,29.58 in,0.0 in,Partly Cloudy,EGPU


,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Station_Code
0,2020-12-01,12:20 AM,41 F,34 F,76 %,WNW,6 mph,0 mph,30.25 in,0.0 in,Fair,EGPU
1,2020-12-01,12:50 AM,41 F,34 F,76 %,W,5 mph,0 mph,30.25 in,0.0 in,Fair,EGPU
2,2020-12-01,1:20 AM,39 F,34 F,81 %,W,5 mph,0 mph,30.25 in,0.0 in,Fair,EGPU


## Combine all stations weather data for year and month

In [ ]:
# Combine the file together
import os, glob
import pandas as pd

path = "/Hertford Uni/Air-PoT/"

def combine_dataset(year,month):
    #all_files = glob.glob(os.path.join(path, "2020-weather data-12-*.csv"))
    all_files = glob.glob(os.path.join(path, year+"-weather data-"+month+"-*.csv"))

    df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
    df_merged   = pd.concat(df_from_each_file, ignore_index=True)
    #df_merged_2020_12.to_csv( "2020_12_merged_weather_data.csv")
    df_merged.to_csv(year+"_"+month+"_merged_weather_data.csv")


In [27]:
len(df_merged_2020_12['Station_Code'].unique())

36

In [28]:
all_files = glob.glob(os.path.join(path, "2021-weather data-1-*.csv"))

df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged_2021_1   = pd.concat(df_from_each_file, ignore_index=True)
df_merged_2021_1.to_csv( "2021_1_merged_weather_data.csv")

In [30]:
len(df_merged_2021_1['Station_Code'].unique())

35

In [29]:
all_files = glob.glob(os.path.join(path, "2021-weather data-2-*.csv"))

df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged_2021_2   = pd.concat(df_from_each_file, ignore_index=True)
df_merged_2021_2.to_csv( "2021_2_merged_weather_data.csv")

In [32]:
df_merged_2021_2.head(3)

,Unnamed: 0,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Station_Code
0,0,2021-02-01,12:20 AM,32 F,32 F,100 %,NE,6 mph,0 mph,28.95 in,0.0 in,Mist,EGBB
1,1,2021-02-01,12:50 AM,34 F,32 F,93 %,NNE,6 mph,0 mph,28.95 in,0.0 in,Mist,EGBB
2,2,2021-02-01,1:20 AM,34 F,32 F,93 %,NE,7 mph,0 mph,28.95 in,0.0 in,Mist,EGBB


In [31]:
len(df_merged_2021_2['Station_Code'].unique())

35

In [33]:
frames = [df_merged_2020_12, df_merged_2021_1, df_merged_2021_2]
whole_weather_data = pd.concat(frames, ignore_index=True)
whole_weather_data.head(3)

,Unnamed: 0,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Station_Code
0,0,2020-12-01,12:20 AM,41 F,36 F,81 %,NW,12 mph,0 mph,29.89 in,0.0 in,Fair,EGBB
1,1,2020-12-01,12:50 AM,41 F,36 F,81 %,NW,10 mph,0 mph,29.89 in,0.0 in,Fair,EGBB
2,2,2020-12-01,1:20 AM,41 F,36 F,81 %,NW,12 mph,0 mph,29.89 in,0.0 in,Fair,EGBB


In [34]:
whole_weather_data.to_csv('whole-weather-data.csv')

In [35]:
whole_weather_data.shape

(86421, 13)

In [36]:
len(whole_weather_data['Station_Code'].unique())

36